# Search Assets

In [ ]:
from dotenv import load_dotenv
import os, sys
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
load_dotenv()

In [ ]:
# Aquasec URL
base_url_nonprod = os.getenv("AQUASEC_NONPROD_URL")
base_url_prod = os.getenv("AQUASEC_PROD_URL")
## Select URL
BASE_URL = base_url_nonprod

In [ ]:
from aquasec_authen import Authentication
# Credentials
username = os.getenv("AQUASEC_USER")
password = os.getenv("AQUASEC_PASSWORD")

# HEADER = Authentication(BASE_URL=BASE_URL).generateCredentialHeader(username=username, password=password)
HEADER = Authentication(BASE_URL=BASE_URL).generateTokenHeader(username=username, password=password)

HEADER

## 1. Search Assets by FreeText

In [ ]:
def extractAsset(asset={}):
    return {
        "id": asset["id"],
        "node_id": asset["node_id"],
        "name": asset["name"],
        "cluster_id": asset["cluster_id"],
        "security_issues": asset["security_issues"]
    }

Maximum page_size is **50**

In [ ]:
def searchAssetsbyFreeText(text="", page=0):
    print("\n[Request] Search Assets by FreeText")
    url = BASE_URL + '/api/v2/infrastructure'
    print("URL  : " + url)

    # max page_size is 50
    param = {
        "search": str(text),
        "page": str(page),
        "page_size": "50",
        "type": "node",
        "order_by": "name",
        "enforced": "true"
    }

    r = requests.get(url, params=param, headers=HEADER)
    if r.status_code == 200:
        r = r.json()
        if r["result"] and r["count"] == len(r["result"]):
            print("\nGetting Successful")
            return [extractAsset(asset=res) for res in r["result"]]
        else:
            print("[Error] Can't Search Assets by FreeText")
            print("        Found Results " + str(len(r["result"])) + " of " + str(r["count"]))
            sys.exit()
    else:
        print("[Error] " + str(r.status_code))
        print("        " + str(r.text))
        sys.exit()

In [ ]:
text = "<Free Text>"

In [ ]:
assetsbyFreeText = searchAssetsbyFreeText(text=text, page=0)

In [ ]:
[[asset["node_id"],asset["name"]] for asset in assetsbyFreeText]

## 2. Search Asset Detail by ID

In [ ]:
def searchAssetDetailbyID(assetID=""):
    print("\n[Request] Search Asset Detail by ID")
    url = BASE_URL + '/api/v2/infrastructure/' + str(assetID)
    print("URL      : " + url)
    print("Asset ID : " + str(assetID))

    r = requests.get(url, headers=HEADER)
    if r.status_code == 200:
        r = r.json()
        print("\nGetting Successful")
        return r
    else:
        print("\n[Error] " + str(r.status_code))
        print("        " + str(r.text))
        sys.exit()

In [ ]:
assetID = "<Asset ID | GUID>"

In [ ]:
assetDetailbyID = searchAssetDetailbyID(assetID=assetID)

In [ ]:
assetDetailbyID

## 3. Search Asset Detail by Name

### assetType
    0 : node
    1 : cluster

In [ ]:
def searchAssetDetailbyName(assetType=0, assetName=""):
    print("\n[Request] Search Asset Detail by Name")

    assetTypes = ["node", "cluster"]
    url = BASE_URL + '/api/v2/infrastructure/' + str(assetTypes[assetType]) + '/' + str(assetName)

    print("URL        : " + url)
    print("Asset Name : " + str(assetName))

    r = requests.get(url, headers=HEADER)
    if r.status_code == 200:
        r = r.json()
        print("\nGetting Successful")
        return r
    else:
        print("\n[Error] " + str(r.status_code))
        print("        " + str(r.text))
        sys.exit()

In [ ]:
assetName="aqua-agent-helm.ip-XXX-XXX-XXX-XXX"

In [ ]:
assetDetailbyName = searchAssetDetailbyName(assetName=assetName)
assetDetailbyName